In [1]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import gzip


In [2]:
# Funzioni per importare file con estensione '.gz'.
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')



# I dataset contengono recensiani di capi di abbigliamento acquistati su ModCloth e RentTheRunway.
ModCloth = getDF('data/modcloth_final_data.json.gz')

# These datasets contain attributes about products sold on ModCloth and Amazon which may be sources of bias in recommendations (in particular, attributes about how the products are marketed). Data also includes user/item interactions for recommendation.
MarketBias = pd.read_csv('data/df_modcloth.csv')

In [3]:
MarketBias.head()

,item_id,user_id,rating,timestamp,size,fit,user_attr,model_attr,category,brand,year,split
0,7443,Alex,4,2010-01-21 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,NaN,2012,0
1,7443,carolyn.agan,3,2010-01-27 08:00:00+00:00,NaN,NaN,NaN,Small,Dresses,NaN,2012,0
2,7443,Robyn,4,2010-01-29 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,NaN,2012,0
3,7443,De,4,2010-02-13 08:00:00+00:00,NaN,NaN,NaN,Small,Dresses,NaN,2012,0
4,7443,tasha,4,2010-02-18 08:00:00+00:00,NaN,NaN,Small,Small,Dresses,NaN,2012,0


In [4]:
MarketBias.shape

(99893, 12)

In [5]:
same = MarketBias[MarketBias.brand == 'ModCloth']

In [6]:
same.shape

(15071, 12)

In [12]:
MarketBias.groupby('item_id').item_id.count()

item_id
6454        66
7443      1011
11960      854
16411     1260
21296     1636
          ... 
155308      10
155317      66
155537      11
155597       7
155950      14
Name: item_id, Length: 1020, dtype: int64

In [17]:
same.head()

,item_id,user_id,rating,timestamp,size,fit,user_attr,model_attr,category,brand,year,split
280,21296,Petra,5,2011-01-25 08:00:00+00:00,NaN,Just right,NaN,Small,Bottoms,ModCloth,2013,1
281,21296,ejs,4,2011-01-26 08:00:00+00:00,NaN,Slightly small,NaN,Small,Bottoms,ModCloth,2013,1
292,21296,Brandi,5,2011-02-02 08:00:00+00:00,NaN,Just right,Small,Small,Bottoms,ModCloth,2013,0
318,21296,Momo,5,2011-02-12 08:00:00+00:00,NaN,Slightly small,Small,Small,Bottoms,ModCloth,2013,0
323,21296,Jackie,5,2011-02-14 08:00:00+00:00,NaN,Just right,Small,Small,Bottoms,ModCloth,2013,0


In [21]:
same.groupby('user_id').rating.mean()

user_id
"Ferrari")     3.0
#              5.0
'Chelle        3.5
-L             5.0
..             5.0
              ... 
zoiekate       4.0
zombie66_79    4.0
zombiscuit     5.0
zumbaneko      4.0
🇦🇺             3.0
Name: rating, Length: 9177, dtype: float64

In [28]:
MarketBias[MarketBias.user_id == "zoiekate"]

,item_id,user_id,rating,timestamp,size,fit,user_attr,model_attr,category,brand,year,split
33331,50405,zoiekate,5,2015-04-16 07:00:00+00:00,1.0,Just right,Small,Small&Large,Tops,NaN,2015,0
59436,73186,zoiekate,5,2016-07-06 07:00:00+00:00,1.0,Just right,Small,Small&Large,Bottoms,NaN,2016,0
75479,146475,zoiekate,3,2017-08-24 07:00:00+00:00,NaN,Very large,Small,Small&Large,Tops,NaN,2018,1
75970,153399,zoiekate,4,2017-09-07 07:00:00+00:00,NaN,Slightly small,Small,Small&Large,Tops,ModCloth,2017,2


In [29]:
ModCloth[ModCloth.user_name == "zoiekate"]

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
37519,416942,NaN,4,5.0,b,NaN,34,new,NaN,5ft 4in,zoiekate,just right,fit,207791,NaN,NaN,Love this sweater. Fit is,Love this sweater. Fit is perfect and goes wit...
38901,424837,NaN,4,4.0,b,NaN,34,tops,NaN,5ft 4in,zoiekate,just right,fit,207791,NaN,NaN,I really love this sweate,I really love this sweater. The color is beaut...
61201,657081,NaN,4,5.0,b,NaN,34,new,NaN,5ft 4in,zoiekate,just right,fit,207791,NaN,NaN,I got the cranberry one a,I got the cranberry one as well and loved it s...
80788,800643,NaN,1,4.0,b,NaN,34,sale,NaN,5ft 4in,zoiekate,just right,fit,207791,NaN,NaN,This is a great blue blaz,This is a great blue blazer! Fits great and qu...
